# NSW Electricity Demand Analysis
## Case Study 1: Daily, Weekly, Seasonal Demand Patterns

This notebook performs exploratory data analysis (EDA) on NSW operational demand data 
to understand daily cycles, weekly behaviour, and seasonal trends.

Dataset Source: AEMO Operational Demand (via Python library or uploaded dataset)
Region: NSW1
Timeframe: YYYY-YYYY (fill after loading data)


In [1]:
# Step 1: Import necessary library files and set up cache
import pandas as pd
from nemosis import dynamic_data_compiler
from datetime import datetime, timedelta
import os
from pymongo import MongoClient

# --- Cache Setup ---
# A temporary directory to store AEMO CSV files downloaded by nemosis
raw_data_cache_dir = "/tmp/nemosis_cache"
os.makedirs(raw_data_cache_dir, exist_ok=True)
print(f"Cache directory ensured: {raw_data_cache_dir}")

# --- AEMO Table Name ---
# We will use DISPATCHREGIONSUM for reliable regional demand data
table_name = "DISPATCHREGIONSUM"

# --- Date range setup ---
start_date = datetime(2024, 1, 1)
end_date = datetime(2025, 1, 1)
print(f"Data range defined from {start_date.date()} to {end_date.date()}")


Cache directory ensured: /tmp/nemosis_cache
Data range defined from 2024-01-01 to 2025-01-01


In [9]:
# Step 2: Connect to MongoDB
try:
    client = MongoClient("mongodb://localhost:27017/")
    # Access the database and collection
    db = client["AEMO_Data_Archive"]
    collection = db["NSW_Operational_Demand_2024"]
    
    # Optional: Check connection by listing existing collections
    print(f"Connected to MongoDB. Collections in 'AEMO_Data_Archive': {db.list_collection_names()}")
    print(f"Target collection is '{collection.name}'.")

except Exception as e:
    print(f"Failed to connect to MongoDB: {e}")
    print("Please ensure your MongoDB service (mongod) is running on localhost:27017.")
    # If connection fails, the script might stop here depending on your environment.


Connected to MongoDB. Collections in 'AEMO_Data_Archive': ['NSW_Operational_Demand_2024']
Target collection is 'NSW_Operational_Demand_2024'.


In [10]:
# Step 3: Loop through data month-by-month and ingest into MongoDB
current = start_date

print("\n--- Starting Data Ingestion Loop ---")

while current < end_date:
    # Calculate the next month's start and end dates accurately
    next_month = (current.replace(day=28) + timedelta(days=4)).replace(day=1)
    end_of_month = next_month - timedelta(seconds=1) 
    
    start_str = current.strftime("%Y/%m/%d %H:%M:%S")
    end_str = end_of_month.strftime("%Y/%m/%d %H:%M:%S")
    
    print(f"\nProcessing {start_str} → {end_str} for {table_name}...")

    try:
        df = dynamic_data_compiler(
            start_str,
            end_str,
            table_name,
            raw_data_cache_dir
        )
        
        # Filter for the NSW region using the verified 'REGIONID' column
        if 'REGIONID' in df.columns:
            df_nsw = df[df["REGIONID"] == "NSW1"].copy()
            
            if not df_nsw.empty:
                # Insert into MongoDB
                records = df_nsw.to_dict("records")
                result = collection.insert_many(records)
                print(f"Inserted {len(result.inserted_ids)} documents for NSW1 in {current.strftime('%B %Y')}.")
            else:
                print(f"No NSW1 specific data found for this period.")
        else:
            print(f"Error: 'REGIONID' column not found in the {table_name} data frame.")

        del df # Release memory
        
    except Exception as e:
        print(f"An error occurred while processing {current.strftime('%B %Y')}: {e}")
        # Continue to the next month even if one month fails
        pass

    current = next_month

print("\n--- Data ingestion complete! ---")


--- Starting Data Ingestion Loop ---

Processing 2024/01/01 00:00:00 → 2024/01/31 23:59:59 for DISPATCHREGIONSUM...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
Inserted 8927 documents for NSW1 in January 2024.

Processing 2024/02/01 00:00:00 → 2024/02/29 23:59:59 for DISPATCHREGIONSUM...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
Inserted 8351 documents for NSW1 in February 2024.

Processing 2024/03/01 00:00:00 → 2024/03/31 23:59:59 for DISPATCHREGIONSUM...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
Inserted 8927 documents for NSW1 in March 2024.

Processing 2024/04/01 00:00:00 → 2024/04/30 23:59:59 for DISPATCHREGIONSUM...
INFO: Compiling data for table DISPATCHREGIONSUM
INFO: Returning DISPATCHREGIONSUM.
Inserted 8639 documents for NSW1 in April 2024.

Processing 2024/05/01 00:00:00 → 2024/05/31 23:59:59 for DISPATCHREGIONSUM...
INFO: Compiling data for table DIS

In [12]:
cursor = collection.find({})
df = pd.DataFrame(list(cursor))

df.head()
df.info()
df.columns


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114335 entries, 0 to 114334
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   _id                          114335 non-null  object        
 1   SETTLEMENTDATE               114335 non-null  datetime64[ns]
 2   REGIONID                     114335 non-null  object        
 3   DISPATCHINTERVAL             114335 non-null  int64         
 4   INTERVENTION                 114335 non-null  int64         
 5   TOTALDEMAND                  114335 non-null  float64       
 6   AVAILABLEGENERATION          114335 non-null  float64       
 7   AVAILABLELOAD                114335 non-null  int64         
 8   DEMANDFORECAST               114335 non-null  int64         
 9   DISPATCHABLEGENERATION       114335 non-null  float64       
 10  DISPATCHABLELOAD             114335 non-null  float64       
 11  NETINTERCHANGE            

Index(['_id', 'SETTLEMENTDATE', 'REGIONID', 'DISPATCHINTERVAL', 'INTERVENTION',
       'TOTALDEMAND', 'AVAILABLEGENERATION', 'AVAILABLELOAD', 'DEMANDFORECAST',
       'DISPATCHABLEGENERATION', 'DISPATCHABLELOAD', 'NETINTERCHANGE',
       'EXCESSGENERATION', 'LOWER5MINLOCALDISPATCH', 'LOWER60SECLOCALDISPATCH',
       'LOWER6SECLOCALDISPATCH', 'RAISE5MINLOCALDISPATCH',
       'RAISE60SECLOCALDISPATCH', 'RAISE6SECLOCALDISPATCH', 'INITIALSUPPLY',
       'CLEAREDSUPPLY', 'LOWERREGLOCALDISPATCH', 'RAISEREGLOCALDISPATCH',
       'TOTALINTERMITTENTGENERATION', 'DEMAND_AND_NONSCHEDGEN', 'UIGF',
       'SEMISCHEDULE_CLEAREDMW', 'SEMISCHEDULE_COMPLIANCEMW'],
      dtype='object')

In [5]:
df.head()

,_id,SETTLEMENTDATE,REGIONID,DISPATCHINTERVAL,INTERVENTION,TOTALDEMAND,AVAILABLEGENERATION,AVAILABLELOAD,DEMANDFORECAST,DISPATCHABLEGENERATION,...,RAISE6SECLOCALDISPATCH,INITIALSUPPLY,CLEAREDSUPPLY,LOWERREGLOCALDISPATCH,RAISEREGLOCALDISPATCH,TOTALINTERMITTENTGENERATION,DEMAND_AND_NONSCHEDGEN,UIGF,SEMISCHEDULE_CLEAREDMW,SEMISCHEDULE_COMPLIANCEMW
0,69211019e0ab8dc0ba7c9c78,2024-12-01 00:05:00,NSW1,20241130241,0,7020.14,10185.12786,238,-38,6834.06,...,53.12,7109.16038,7068.98,73.0,6.0,54.77584,7123.75584,446.12786,446.12786,0.0
1,69211019e0ab8dc0ba7c9c79,2024-12-01 00:10:00,NSW1,20241130242,0,7049.46,10179.59995,238,-36,6823.54,...,65.00,7126.41431,7095.58,90.0,6.0,54.24489,7149.82489,440.59995,440.59995,0.0
2,69211019e0ab8dc0ba7c9c7a,2024-12-01 00:15:00,NSW1,20241130243,0,6996.36,10182.24625,238,-36,6838.43,...,105.00,7070.87424,7080.85,38.0,6.0,50.97204,7131.82204,444.24625,444.24625,0.0
3,69211019e0ab8dc0ba7c9c7b,2024-12-01 00:20:00,NSW1,20241130244,0,6924.10,10181.32000,238,-36,6739.68,...,105.00,7034.00420,7044.83,32.0,17.0,48.04622,7092.87622,443.32000,443.32000,0.0
4,69211019e0ab8dc0ba7c9c7c,2024-12-01 00:25:00,NSW1,20241130245,0,6894.04,10156.90116,238,-37,6735.41,...,103.16,7031.71053,7063.10,32.0,21.0,49.87204,7112.97204,418.90116,418.90116,0.0


In [6]:
df_analysis = df[['SETTLEMENTDATE', 'TOTALDEMAND']]



In [13]:
df_analysis.head()

,SETTLEMENTDATE,TOTALDEMAND
0,2024-12-01 00:05:00,7020.14
1,2024-12-01 00:10:00,7049.46
2,2024-12-01 00:15:00,6996.36
3,2024-12-01 00:20:00,6924.10
4,2024-12-01 00:25:00,6894.04


In [15]:
df_analysis['SETTLEMENTDATE'] = pd.to_datetime(df['SETTLEMENTDATE'])

df_analysis['hour'] = df['SETTLEMENTDATE'].dt.hour
df_analysis['day'] = dfdf['SETTLEMENTDATE'].dt.day
df_analysis['weekday'] = df['SETTLEMENTDATE'].dt.day_name()
df_analysis['month'] = df['SETTLEMENTDATE'].dt.month
df_analysis['month_name'] = df['SETTLEMENTDATE'].dt.month_name()
df_analysis['date'] = df['SETTLEMENTDATE'].dt.date


NameError: name 'dfdf' is not defined

In [ ]:
df_analysis.head()

In [ ]:
df_analysis['season'] = df_analysis['SETTLEMENTDATE'].dt.month % 12 // 3 + 1
df_analysis['season_name'] = df_analysis['season'].map({
    1: 'Summer',
    2: 'Autumn',
    3: 'Winter',
    4: 'Spring'
})


In [ ]:
df_analysis.head()

In [ ]:
df_analysis.info()